In [27]:
# This is Derived from 
# https://github.com/gkamradt/langchain-tutorials/blob/main/data_generation/5%20Levels%20Of%20Summarization%20-%20Novice%20To%20Expert.ipynb

# Workaround OpenAI's Token Limit
# https://www.youtube.com/watch?v=f9_BWhCI4Zo&t=418s

In [ ]:
!pip install langchain
!pip install python-dotenv


In [7]:

%load_ext dotenv
%dotenv


The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [35]:
%env OPENAI_KEY=''

env: OPENAI_KEY=''


In [24]:
import os
from dotenv import load_dotenv
from langchain import OpenAI
from langchain import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [ ]:
import os 
open_api_key = os.environ["OPENAI_KEY"]
print(open_api_key)



In [22]:
llm = OpenAI( temperature=0, openai_api_key= '')

In [13]:
# Level 0 try
prompt = """
Please provide a summary of the following text

TEXT:
Philosophy (from Greek: φιλοσοφία, philosophia, 'love of wisdom') \
is the systematized study of general and fundamental questions, \
such as those about existence, reason, knowledge, values, mind, and language. \
Some sources claim the term was coined by Pythagoras (c. 570 – c. 495 BCE), \
although this theory is disputed by some. Philosophical methods include questioning, \
critical discussion, rational argument, and systematic presentation.
"""
num_tokens = llm.get_num_tokens(prompt)
print( num_tokens)

121


In [23]:
output = llm(prompt)
print(output)


Philosophy is a systematized study of general and fundamental questions about existence, reason, knowledge, values, mind, and language. It is believed to have been coined by Pythagoras, and its methods include questioning, critical discussion, rational argument, and systematic presentation.


In [25]:
#This is Level 1 - Read large data
with open('data/PivotPod_412.txt', 'r') as file: 
    file_contents = file.read()
    print(file_contents[:300])

support for pivot comes from service now today it seems every business is influenced by the uncertainty in the world it seeps into every discussion it forces tough choices for what to do next should you drive efficiency or should you drive growth but what if instead of choosing you could just say ye


In [26]:
llm.get_num_tokens(file_contents)

14557

In [29]:
# 1 token = 4 characters
# 3000 tokens = 4 *  3000 
text_splitter = RecursiveCharacterTextSplitter( separators=["\n\n", "\n", " "], chunk_size=12000, chunk_overlap=500)
docs = text_splitter.create_documents( [ file_contents])

In [32]:
num_docs = len(docs)
num_tokens_first_doc = llm.get_num_tokens(docs[0].page_content)
print(" num docs" , num_docs)
print(" num tokens ", num_tokens_first_doc)


 num docs 7
 num tokens  2438


In [34]:
summary_chain = load_summarize_chain( llm=llm, chain_type='map_reduce')
output = summary_chain.run(docs)
print(output)

 George Soros is handing control of his $25 billion empire to his son Alex, while Twitter CEO Linda Yakarino and Elon Musk's recent tweet of a woman's boobs have made her job more difficult. Donald Trump has been federally indicted for the first time in history, and Tesla has recently struck agreements with GM and Ford to access Tesla's 17,000 supercharger plugs. Jennifer Parker and Jennifer Pahlka have both been involved in improving government services, while Brooklyn and Linens provides luxury bedding and the podcast Into the Mix explores global refugee policy. Fred Ryan is stepping down as publisher and chief executive of the Washington Post, and Lionel Messi chose to move to Miami instead of taking money from Saudi Arabia. This episode of Pivot from New York Magazine featured a discussion about the great state of Florida and ZipRecruiter and Clean My Mac X were featured.
